In [2]:
import os

# change the working directory to the location of files
os.chdir('D:/eo_data/Ibague/')
print(os.getcwd())
# store the files list to a variable
eo_files = os.listdir(os.getcwd())
print(eo_files)

D:\eo_data\Ibague
['data', 'S1_GRD', 'S2_L1C']


In [18]:
# Unzip files
import zipfile

# Check if a data folder exist
#if not os.path.exists('data'):
#    os.makedirs('data')
#    print 'data folder' + ' was created'
# Check if the name of the data folder is in
#if 'data' in eo_files:
#    eo_files.remove('data')
##TODO catch error when a directory is in eo_files

def unzipfiles(path):
    """
    Unzips every zipfile in the path, and stores in directory with zipfile name+.SAFE
    Args:
        path (string): string of directory where zipfiles are located 
    """
    for im_id in os.listdir(path):
        if not os.path.exists('data/'+im_id[:-3]+'SAFE'):
            print('Unzipping ' + im_id)
            zip_ref = zipfile.ZipFile(path+im_id, 'r')
            zip_ref.extractall('data')
            zip_ref.close()
        else:
            print(im_id[:-4] + ' was already uncompressed')

In [19]:
os.getcwd()
unzipfiles("D:/eo_data/Ibague/S2_L1C/")

S2A_MSIL1C_20171021T153111_N0205_R025_T18NVL_20171021T153112 was already uncompressed
S2A_MSIL1C_20171210T152631_N0206_R025_T18NVK_20171210T170838 was already uncompressed
S2A_MSIL1C_20171220T153111_N0206_R025_T18NVK_20171220T170223 was already uncompressed
S2A_MSIL1C_20171220T153111_N0206_R025_T18NVL_20171220T170223 was already uncompressed
S2B_MSIL1C_20171016T153059_N0205_R025_T18NVK_20171016T153058 was already uncompressed
S2B_MSIL1C_20171016T153059_N0205_R025_T18NVL_20171016T153058 was already uncompressed
S2B_MSIL1C_20171105T153059_N0206_R025_T18NVK_20171105T170109 was already uncompressed
S2B_MSIL1C_20171105T153059_N0206_R025_T18NVL_20171105T170109 was already uncompressed
S2B_MSIL1C_20171205T152629_N0206_R025_T18NVK_20171205T170944 was already uncompressed
S2B_MSIL1C_20171205T152629_N0206_R025_T18NVL_20171205T170944 was already uncompressed
S2B_MSIL1C_20171215T152629_N0206_R025_T18NVK_20171215T201906 was already uncompressed
S2B_MSIL1C_20171215T152629_N0206_R025_T18NVL_20171215T